In [1]:
from bs4 import BeautifulSoup
import requests

In [3]:
import pandas as pd

In [23]:
import time
import random

In [35]:
# Abre el archivo 'cities' y lee su contenido
with open(r'tests\cities.csv', 'r') as file:
    # Lee todas las líneas del archivo y elimina los saltos de línea
    cities_list = [line.strip() for line in file]

# Imprime la lista de ciudades
print(cities_list)

['Altamonte-Springs', 'Apopka', 'Bartow', 'Celebration', 'Clermont', 'Dunnellon', 'Eatonville', 'Kissimmee', 'Lake-Buena-Vista', 'Lake-Mary', 'Lake-Placid', 'Lake-Wales', 'Lakeland', 'Mount-Dora', 'Ocala', 'Orlando', 'Oviedo', 'Sanford', 'Sebring', 'St-Cloud', 'Tavares', 'Umatilla', 'Wauchula', 'Wildwood', 'Winter-Garden', 'Winter-Haven', 'Winter-Park']


In [36]:
data_list = []

# Itera sobre cada ciudad en la lista
for city in cities_list:
    # URL de la página web para la ciudad actual
    url = f'https://www.visitflorida.com/places-to-go/central/{city}/'
    
    # Realiza una solicitud HTTP a la página web
    response = requests.get(url)
    
    # Verifica que la solicitud fue exitosa
    if response.status_code == 200:
        # Analiza el contenido HTML de la página
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encuentra los elementos con la clase 'left-container sv-col'
        elements = soup.find_all(class_='left-container sv-col')
        
        # Extrae y almacena el contenido de esos elementos en un diccionario
        for element in elements:
            data = {
                'city': city,
                'content': element.get_text(strip=True)
            }
            data_list.append(data)
    else:
        print(f'Error al acceder a la página de {city}: {response.status_code}')
    
    # Espera un tiempo aleatorio entre 1 y 5 segundos antes de la siguiente solicitud
    time.sleep(random.uniform(1, 5))


In [37]:
print(data_list[:3])

[{'city': 'Altamonte-Springs', 'content': 'Only 15 minutes north of downtownOrlando, 30 minutes north ofWalt Disney World, and less than an hour fromDaytona Beach, the city ofAltamonte Springsis conveniently located nearCentral Florida’smajor attractions while offering plenty to see and do within the city.A wide variety of shopping and dining choices can be found at the business and urban residential district ofUptown Altamonte, home to theAltamonte Mall. Its focal point, lakefrontCranes Roost Park, features anamphitheater, a61-jet fountainthat displays choreographed shows, aEuropean-style plaza, and a19-foot-tall steel crane sculpture. At the 45-acre park, visitors can stroll or jog along the waterfront,fish, picnic, and attend specialevents, which includeBattle of the Bands,Red Hot & Boom, andLight up the Holidays.Altamonte Springs also offers a cutting-edge way to get around town withCraneRIDES, its autonomous vehicle shuttle. The service runs through Uptown Altamonte, stopping at A

In [38]:
import json

In [39]:
ruta_archivo = 'tests/data.json'

In [40]:
with open(ruta_archivo, "w", encoding="utf-8") as archivo:
    json.dump(data_list, archivo, ensure_ascii=False, indent=4)